In [76]:
import pandas as pd
import numpy as np
# Okay Looking at my data --> I have genres, tags, popularity and year proximity. I can use all of these to create a recommendation system.

In [77]:
df = pd.read_csv('/Users/mihirparikh/Documents/GitHub/Movie-Recs-Algo/MovieLens Dataset/movies.csv')
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [79]:
df2 = pd.read_csv('/Users/mihirparikh/Documents/GitHub/Movie-Recs-Algo/MovieLens Dataset/tags.csv')
df2.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [80]:
df3 = pd.read_csv('/Users/mihirparikh/Documents/GitHub/Movie-Recs-Algo/MovieLens Dataset/ratings.csv')
df3.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [81]:
## Steps:
## movies.csv: Create a year column and then normalize the year column - done 
## tags.csv: Concat the multiple tabs into one column and then pull it into movies.csv 
## ratings.csv: Group the ratings by movie --> Average + Count of ratings.  

In [82]:
df["year"] = df['title'].str.extract(r"\((\d{4})\)").astype(float)

In [83]:
df2

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [84]:
def group_tags(df, id_col="movieId", tag_col="tag"):
    """
    Groups a dataframe by `id_col` and concatenates the values
    in `tag_col` into a single string separated by '|'.
    """
    grouped = (
        df.groupby(id_col)[tag_col]
          .apply(lambda x: "|".join(x.astype(str)))
          .reset_index()
    )
    return grouped


In [85]:
tags_grouped = group_tags(df2)
print(tags_grouped)

      movieId                                              tag
0           1                                  pixar|pixar|fun
1           2     fantasy|magic board game|Robin Williams|game
2           3                                        moldy|old
3           5                                 pregnancy|remake
4           7                                           remake
...       ...                                              ...
1567   183611                      Comedy|funny|Rachel McAdams
1568   184471  adventure|Alicia Vikander|video game adaptation
1569   187593                Josh Brolin|Ryan Reynolds|sarcasm
1570   187595                          Emilia Clarke|star wars
1571   193565                    anime|comedy|gintama|remaster

[1572 rows x 2 columns]


In [86]:
df_semiprepped = df.merge(tags_grouped, on="movieId", how="left")

In [87]:
df_semiprepped.head()

,movieId,title,genres,year,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,pixar|pixar|fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,fantasy|magic board game|Robin Williams|game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,moldy|old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,NaN
4,5,Father of the Bride Part II (1995),Comedy,1995.0,pregnancy|remake


In [88]:
df3.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [89]:
def group_ratings(df, id_col="movieId", rating_col="rating"):
    """
    Groups dataframe by movieId and computes:
      - average rating
      - count of ratings
    Returns a new dataframe with movieId, avg_rating, and rating_count.
    """
    grouped = (
        df.groupby(id_col)[rating_col]
          .agg(avg_rating="mean", rating_count="count")
          .reset_index()
    )
    grouped["avg_rating"] = grouped["avg_rating"].round(2)
    return grouped

In [90]:
ratings_grouped = group_ratings(df3)
ratings_grouped.head()

,movieId,avg_rating,rating_count
0,1,3.92,215
1,2,3.43,110
2,3,3.26,52
3,4,2.36,7
4,5,3.07,49


In [91]:
df_semiprepped_w_ratings = df_semiprepped.merge(ratings_grouped, on="movieId", how="left")

In [92]:
df_semiprepped_w_ratings

,movieId,title,genres,year,tag,avg_rating,rating_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,pixar|pixar|fun,3.92,215.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,fantasy|magic board game|Robin Williams|game,3.43,110.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,moldy|old,3.26,52.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,NaN,2.36,7.0
4,5,Father of the Bride Part II (1995),Comedy,1995.0,pregnancy|remake,3.07,49.0
...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017.0,NaN,4.00,1.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017.0,NaN,3.50,1.0
9739,193585,Flint (2017),Drama,2017.0,NaN,3.50,1.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018.0,NaN,3.50,1.0


In [93]:
## Okay so now I have a dataframe with (1) Title (2) Genres (3) Year (4) Tags (5) Average Rating (6) Number of Ratings
## Now i can start encoding the data.
## Year encoded - done
## 

In [94]:
df_semiprepped_w_ratings['year'] = df_semiprepped_w_ratings['year'].astype("Int64")
min_year, max_year = df_semiprepped_w_ratings["year"].min(), df_semiprepped_w_ratings["year"].max().astype(int)
df_semiprepped_w_ratings["year_scaled"] = (df_semiprepped_w_ratings["year"] - min_year) / (max_year - min_year)

In [95]:
df_semiprepped_w_ratings["year_scaled"].nunique()

106

In [96]:
def normalize_tags_and_genres(df, target_col):
    """
    Lowercase, trim whitespace, and re-join tags consistently with '|'.
    """
    return (
        df[target_col]
        .fillna("")
        .astype(str)
        .str.lower()                 # make everything lowercase
        .str.replace(r"\s+", " ", regex=True)  # normalize multiple spaces
        .str.strip()
        .str.split("|")
        .apply(lambda target_col: "|".join(sorted(set(t.strip() for t in target_col if t))))
    )

In [97]:
df_semiprepped_w_ratings['tag'] = normalize_tags_and_genres(df_semiprepped_w_ratings, 'tag')
df_semiprepped_w_ratings['genres'] = normalize_tags_and_genres(df_semiprepped_w_ratings, 'genres')


In [98]:
genres_ohe = df_semiprepped_w_ratings["genres"].str.get_dummies(sep="|").astype(int)
genres_ohe = genres_ohe.add_prefix("genre::")

In [99]:
genres_ohe.head()
## Okay so what I have here is all possible combinations of genres 

,genre::(no genres listed),genre::action,genre::adventure,genre::animation,genre::children,genre::comedy,genre::crime,genre::documentary,genre::drama,genre::fantasy,genre::film-noir,genre::horror,genre::imax,genre::musical,genre::mystery,genre::romance,genre::sci-fi,genre::thriller,genre::war,genre::western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [100]:
genres_encoded = pd.concat([df_semiprepped_w_ratings, genres_ohe], axis=1)

In [101]:
tag_ohe = genres_encoded["tag"].str.get_dummies(sep="|").astype(int)

# add a prefix so we know these are genre features
tag_ohe = tag_ohe.add_prefix("tag::")

# merge back
ohe_encoding_complete = pd.concat([genres_encoded, tag_ohe], axis=1)



In [102]:
ohe_encoding_complete.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9742 entries, 0 to 9741
Data columns (total 1503 columns):
 #     Column                                                                                      Dtype  
---    ------                                                                                      -----  
 0     movieId                                                                                     int64  
 1     title                                                                                       object 
 2     genres                                                                                      object 
 3     year                                                                                        Int64  
 4     tag                                                                                         object 
 5     avg_rating                                                                                  float64
 6     rating_count                

In [103]:
def count_tag_occurrences(df, tags_col="tag", tag="will ferrell"):
    """
    Count how many rows in df[tags_col] contain the given tag 
    (case-insensitive, works with '|' separated tags).
    """
    tag_lower = tag.lower()
    return (
        df[tags_col]
        .fillna("")
        .str.lower()
        .str.split("|")
        .apply(lambda tags: tag_lower in [t.strip() for t in tags])
        .sum()
    )

def sum_ohe_column(df, col_name="tag::will ferrell"):
    """
    Sum the one-hot encoded column (number of movies that had this tag).
    """
    if col_name in df.columns:
        return df[col_name].sum()
    else:
        raise ValueError(f"Column {col_name} not found in dataframe")

In [104]:
raw_count = count_tag_occurrences(df_semiprepped, tags_col="tag", tag="will ferrell")
print("Count from raw tags:", raw_count)

# Sum in one-hot encoded version
ohe_count = sum_ohe_column(ohe_encoding_complete, col_name="tag::will ferrell")
print("Count from one-hot column:", ohe_count)


Count from raw tags: 6
Count from one-hot column: 6


In [105]:
def movies_with_tag(df, tags_col="tag", title_col="title", tag="will ferrell"):
    """
    Return the subset of rows where the pipe-separated `tags_col`
    contains the given `tag` (case-insensitive).
    """
    tag_lower = tag.lower()
    mask = (
        df[tags_col]
        .fillna("")
        .str.lower()
        .str.split("|")
        .apply(lambda tags: tag_lower in [t.strip() for t in tags])
    )
    return df.loc[mask, [title_col, tags_col]]

movies_with_tag(df_semiprepped, tags_col="tag", title_col="title", tag="will ferrell")



,title,tag
3525,Zoolander (2001),ben stiller|comedy|David Bowie|goofy|mindless ...
4245,Old School (2003),comedy|Will Ferrell
5263,Anchorman: The Legend of Ron Burgundy (2004),hilarious|Steve Carell|Will Ferrell|stupid|awe...
6256,Stranger than Fiction (2006),emma thompson|Maggie Gyllenhaal|modern fantasy...
6801,Step Brothers (2008),funny|Highly quotable|will ferrell|comedy|funn...
8321,Anchorman 2: The Legend Continues (2013),comedy|Steve Carell|stupid but funny|will ferrell


In [107]:
def verify_tag_encoding(df_raw, ohe_df, tags_col="tag", prefix="tag::"):
    """
    Compare raw tag frequencies vs. one-hot encoded column sums.

    Returns a DataFrame with columns:
      - tag
      - raw_count  (from text column)
      - ohe_count  (from one-hot columns)

    Assumes tags in df_raw[tags_col] are pipe-separated and already normalized
    (lowercase, trimmed). If not, run `normalize_tags` first.
    """
    # Count tags from raw text
    raw_exploded = (
        df_raw[tags_col]
        .fillna("")
        .str.split("|")
        .explode()
        .str.strip()
    )
    raw_counts = raw_exploded[raw_exploded != ""].value_counts()

    # Count from one-hot columns (sum over each column)
    ohe_counts = ohe_df.filter(like=prefix).sum().astype(int)
    ohe_counts.index = ohe_counts.index.str.replace(prefix, "")

    # Merge both counts into one DataFrame
    comparison = (
        pd.DataFrame({
            "raw_count": raw_counts,
            "ohe_count": ohe_counts
        })
        .fillna(0)
        .astype(int)
        .sort_index()
    )

    return comparison


In [109]:
comparison = verify_tag_encoding(df_semiprepped_w_ratings, ohe_encoding_complete, tags_col="tag", prefix="tag::")
print(comparison.head(10))

                                           raw_count  ohe_count
"artsy"                                            1          1
06 oscar nominated best movie - animation          3          3
1900s                                              1          1
1920s                                              2          2
1950s                                              2          2
1960s                                              1          1
1970s                                              3          3
1980s                                              2          2
1990s                                              1          1
2001-like                                          1          1


In [ ]:
## Okay what do I have now? 
## I have (1) Genres Encoded, (2) Tags Encoded, (3) Year Scaled, (4) Average Rating, (5) Number of Ratings
## Turns out I dont need Years Scale since some